In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('dataset_before_label.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9091 entries, 0 to 9090
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   uniqueId    9091 non-null   object 
 1   clean_text  9091 non-null   object 
 2   rep_score   9091 non-null   float64
dtypes: float64(1), object(2)
memory usage: 213.2+ KB


In [3]:
df.head(10)

,uniqueId,clean_text,rep_score
0,l.galang146,makasar butuh 1 hari masuk dpr bandung butuh 2...,0.115
1,ucuprevan,datangin puan,0.182
2,emiaasli,bantu up teman',0.356
3,liezjameelahamzah,jangan dibakar jangan dirusak kak. diperbaikin...,0.157
4,makichen06,"dpr nya enggak kerja,kasus vina saja enggak bi...",0.149
5,meicia2020,"gue capek bilang ini, demo kita sekarang sudah...",0.415
6,ayyyuraaa_,tolong sebarin bro,0.290
7,cliper_pemula36,sebernya bukan itu tujuan nya. aspirasinya yan...,0.162
8,dika.dch,guys semua live di t hilang,0.199
9,yanrvnhrs,fitur live dimatiin guys,0.296


In [4]:
initial_data = df.head(50)
initial_data.info(), display(initial_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   uniqueId    50 non-null     object 
 1   clean_text  50 non-null     object 
 2   rep_score   50 non-null     float64
dtypes: float64(1), object(2)
memory usage: 1.3+ KB


,uniqueId,clean_text,rep_score
0,l.galang146,makasar butuh 1 hari masuk dpr bandung butuh 2...,0.115
1,ucuprevan,datangin puan,0.182
2,emiaasli,bantu up teman',0.356
3,liezjameelahamzah,jangan dibakar jangan dirusak kak. diperbaikin...,0.157
4,makichen06,"dpr nya enggak kerja,kasus vina saja enggak bi...",0.149


(None, None)

# Initial Try

In [3]:
import google.generativeai as genai
import json
import time

In [ ]:
API_KEY = "API-KEY"
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-flash-latest")

In [5]:
initial_prompt = """
Anda adalah seorang Social Media Analyst yang ahli dalam mendeteksi pola aktivitas inautentik (buzzer) pada platform TikTok.
Tugas Anda adalah mengklasifikasikan komentar pengguna ke dalam kategori Buzzer atau Non-Buzzer berdasarkan teks komentar dan skor repetisi (repetitive score) yang diberikan.

KONTEKS PERISTIWA (PENTING UNTUK DIPAHAMI):
Komentar-komentar ini diambil dari video TikTok mengenai demonstrasi buruh dan isu gaji DPR.
- Kronologi Situasi: Awalnya aksi berjalan kondusif, namun berubah menjadi rusuh/anarkis (termasuk pembakaran halte).
- Pemicu Eskalasi: Situasi memanas setelah terjadi insiden mobil taktis polisi menabrak seorang driver ojek online (Gojek).
- Fokus Analisis: Harap bedakan antara reaksi emosional alami masyarakat terhadap insiden tersebut dengan narasi yang sengaja dipolakan (buzzer) untuk memprovokasi keadaan atau justru mengalihkan isu (penggiringan opini).

Anda HARUS dan HANYA mengembalikan data dalam format JSON yang valid, tanpa memberikan penjelasan tambahan apapun sebelum ataupun sesudah JSON tersebut. Berikut adalah format JSON yang harus Anda ikuti:
JSON Anda harus memiliki struktur key dan value sebagai berikut:
1. "classification": (string)
2. "reasoning": (string - ringkasan singkat alasan pemilihan kategori dikaitkan dengan konteks peristiwa)
---
ATURAN untuk key "classification":
Gunakan HANYA SATU dari tiga nilai berikut berdasarkan kombinasi Teks dan Repetitive Score:

a. "POSITIVE_BUZZER"
   - KRITERIA TEKS: Komentar bersifat formulaik (berpola baku), respons berulang, menggunakan bahasa emosional yang tegas/memuji berlebihan (misal: membela aparat secara membabi buta atau menyalahkan demonstran dengan narasi seragam), atau ekspresi yang memicu polarisasi diskusi.
   - KRITERIA SKOR: Memiliki Repetitive Score yang CENDERUNG TINGGI.

b. "NEGATIVE_BUZZER"
   - KRITERIA TEKS: Memiliki ciri linguistik yang sama dengan Positive Buzzer (formulaik, repetitif, emosional), TETAPI menyampaikan sentimen negatif, menyerang, atau menentang (misal: spamming provokasi anarkis, ujaran kebencian berulang terhadap institusi tanpa argumen personal).
   - KRITERIA SKOR: Memiliki Repetitive Score yang CENDERUNG TINGGI.

c. "NON_BUZZER"
   - KRITERIA TEKS: Komentar terdengar personal, kritis, reflektif (misal: simpati tulus pada driver Gojek), sarkastik, humoris, ironis, atau bersifat evaluatif/bertanya. Struktur kalimat terlihat organik dan unik.
   - KRITERIA SKOR: Memiliki Repetitive Score yang CENDERUNG RENDAH.

---
ATURAN LOGIKA PENTING (WAJIB DIKUTI):
1. Perhatikan **Repetitive Score**:
   - Semakin TINGGI skor (mendekati 1.0), semakin besar probabilitas komentar tersebut adalah "POSITIVE_BUZZER" atau "NEGATIVE_BUZZER".
   - Semakin RENDAH skor, semakin besar probabilitas komentar tersebut adalah "NON_BUZZER".
2. Gunakan konteks peristiwa (Demo, Gaji DPR, Insiden Gojek, Bakar Halte) untuk memahami nuansa teks. Jika teks terlihat repetitif dan provokatif tentang topik ini, pertimbangkan label Buzzer.
3. Jika teks terlihat sangat organik/unik (misal: curhat personal pengalaman di lokasi demo) meskipun skor repetisinya sedang, prioritaskan klasifikasi "NON_BUZZER".
4. Pastikan JSON yang dihasilkan valid.

---

Sekarang, analisis data berikut dan kembalikan hasilnya dalam format JSON yang telah ditentukan:
Ulasan : "{clean_text}"
Repetitive Score : {rep_score}

JSON:
"""

In [15]:
import os
import json
import time
import tempfile
from tqdm import tqdm
import pandas as pd

# --- user inputs / existing objects ---
# full_data should exist (your original df)
full_data = df.copy()
full_data = full_data[5001:]

checkpoint_filename = 'checkpoint_labelling.csv'
final_output_filename = 'full_labelling_FINAL.csv'

# results and start index
result2 = []
start_index = 0

# --- helper: safe atomic save (optional but recommended) ---
def safe_to_csv(df_to_save, target_path):
    dirpath = os.path.dirname(os.path.abspath(target_path)) or '.'
    fd, tmp_path = tempfile.mkstemp(prefix='tmp_', suffix='.csv', dir=dirpath)
    os.close(fd)
    try:
        df_to_save.to_csv(tmp_path, index=False)
        os.replace(tmp_path, target_path)  # atomic replace
        return True
    except Exception as e:
        try:
            os.remove(tmp_path)
        except Exception:
            pass
        print("Error while saving file:", e)
        return False

# --- 1) Resume from checkpoint if it exists ---
if os.path.exists(checkpoint_filename):
    print(f"Found checkpoint file: {checkpoint_filename} — resuming...")
    try:
        df_checkpoint = pd.read_csv(checkpoint_filename)
        # expected columns: 'ai_classification_v1', 'ai_reasoning_v1'
        if 'ai_classification_v1' in df_checkpoint.columns and 'ai_reasoning_v1' in df_checkpoint.columns:
            # populate result2 as list of dicts (same shape as you'd append)
            result2 = df_checkpoint[['ai_classification_v1', 'ai_reasoning_v1']].to_dict('records')
            start_index = len(result2)
            print(f"Resuming from row {start_index} (already processed).")
        else:
            print("Checkpoint exists but missing expected AI columns — starting from 0.")
            result2 = []
            start_index = 0
    except Exception as e:
        print("Failed to read checkpoint — starting from 0. Error:", e)
        result2 = []
        start_index = 0
else:
    print("No checkpoint found — starting from 0.")
    result2 = []
    start_index = 0

# how many remain
remaining = full_data.shape[0] - start_index
print(f"Will process {remaining} rows (from index {start_index} to {full_data.shape[0]-1}).")

# --- prepare subset to process (skip already processed) ---
data_to_process = full_data.iloc[start_index:].copy()

# --- robust extractor function (your previous one) ---
def extract_text_from_response(response):
    # try response.text first
    try:
        return response.text
    except Exception:
        pass
    if hasattr(response, 'output_text') and response.output_text:
        return response.output_text
    try:
        if hasattr(response, 'candidates') and response.candidates:
            cand = response.candidates[0]
            if hasattr(cand, 'text') and cand.text:
                return cand.text
            if hasattr(cand, 'content') and cand.content:
                parts = cand.content
                texts = []
                for p in parts:
                    if isinstance(p, str):
                        texts.append(p)
                    elif isinstance(p, dict):
                        texts.append(p.get('text') or p.get('content') or json.dumps(p))
                    else:
                        texts.append(str(p))
                return ''.join(filter(None, texts))
    except Exception:
        pass
    if hasattr(response, 'output') and response.output:
        try:
            out = response.output
            if isinstance(out, (list, tuple)):
                return ''.join(str(x) for x in out)
            return str(out)
        except Exception:
            pass
    return str(response)

# --- MAIN LOOP: iterate only over remaining rows ---
for local_idx, (index, row) in enumerate(tqdm(data_to_process.iterrows(), total=data_to_process.shape[0])):
    # local_idx is 0..n-1 for the chunk being processed
    text = row['clean_text']
    repetitive_score = row.get('rep_score', None)  # in case column missing
    prompt_final = initial_prompt.format(clean_text=text, rep_score=repetitive_score)

    response = None  # ensure defined in except block
    try:
        # call your model
        response = model.generate_content(prompt_final)

        # extract text robustly
        raw_output = extract_text_from_response(response)
        if raw_output is None or len(raw_output.strip()) == 0:
            print("DEBUG: no text returned from model for row", start_index + local_idx)
            print("Raw response repr:", repr(response))
            raise ValueError("No textual parts found in response")

        # clean & parse JSON
        clean_json_string = raw_output.strip().replace("```json", "").replace("```", "")
        start_idx_json = clean_json_string.find("{")
        end_idx_json = clean_json_string.rfind("}") + 1

        if start_idx_json == -1 or end_idx_json == -1:
            snippet = clean_json_string[:500]
            raise ValueError(f"JSON not found in model output. Snippet: {snippet!r}")

        json_string = clean_json_string[start_idx_json:end_idx_json]
        data = json.loads(json_string)

        clean_data = {
            'ai_classification_v1': data.get('classification'),
            'ai_reasoning_v1': data.get('reasoning')
        }
        result2.append(clean_data)

    except Exception as e:
        # gather debug extras (may be None)
        extra = {}
        try:
            extra['finish_reason'] = getattr(response, 'finish_reason', None)
        except Exception:
            extra['finish_reason'] = None
        try:
            extra['candidates'] = getattr(response, 'candidates', None)
        except Exception:
            extra['candidates'] = None

        print(f"--- ERROR processing global row {start_index + local_idx} (orig idx {index}) ---")
        print("Error:", e, "; extra:", extra)
        result2.append({
            'ai_classification_v1': 'ERROR',
            'ai_reasoning_v1': f'ERROR: {str(e)}; extra={extra}'
        })

    # be polite to API
    time.sleep(4)

    # --- CHECKPOINT: every 50 total results (including previously loaded ones) ---
    if len(result2) % 50 == 0:
        df_temp_result = pd.DataFrame(result2)
        # take first len(result2) rows from original data (these are processed)
        current_processed_data = full_data.iloc[:len(result2)].reset_index(drop=True)
        df_checkpoint_save = pd.concat([current_processed_data, df_temp_result], axis=1)

        # try safe save, fallback to normal save on failure
        saved = safe_to_csv(df_checkpoint_save, checkpoint_filename)
        if not saved:
            try:
                df_checkpoint_save.to_csv(checkpoint_filename, index=False)
                saved = True
            except Exception as e:
                print("Failed to write checkpoint (both atomic and direct). Error:", e)
                saved = False

        if saved:
            print(f" [CHECKPOINT] Data safely saved up to row {len(result2)}")
        else:
            print(" [CHECKPOINT] Failed to save checkpoint. Check file locks/permissions.")

print("Analysis Completed!")

# --- FINAL SAVE ---
df_final_result = pd.DataFrame(result2)
df_final_segmented = pd.concat([full_data.reset_index(drop=True), df_final_result], axis=1)

# final save (use safe atomic write)
if not safe_to_csv(df_final_segmented, final_output_filename):
    # fallback
    df_final_segmented.to_csv(final_output_filename, index=False)

print(f"Success, saved final file to : {final_output_filename}")
display(df_final_segmented.head())

Found checkpoint file: checkpoint_labelling.csv — resuming...
Resuming from row 3850 (already processed).
Will process 240 rows (from index 3850 to 4089).


 21%|██        | 50/240 [06:29<24:53,  7.86s/it]

 [CHECKPOINT] Data safely saved up to row 3900


 42%|████▏     | 100/240 [12:51<17:08,  7.35s/it]

 [CHECKPOINT] Data safely saved up to row 3950


...

 83%|████████▎ | 200/240 [25:51<05:14,  7.85s/it]

 [CHECKPOINT] Data safely saved up to row 4050


100%|██████████| 240/240 [31:01<00:00,  7.76s/it]


Analysis Completed!
Success, saved final file to : full_labelling_FINAL.csv


,uniqueId,clean_text,rep_score,ai_classification_v1,ai_reasoning_v1
0,uceng_wader,kasihan warga sipil. selalu di benturkan denga...,0.121,NaN,NaN
1,liliswati94,"ya allah , selamat kan lah dan lindungi lah pa...",0.166,NaN,NaN
2,batuguntjngkertas,#stoppbayarpajakditahun2025 kenapa??? kalo eng...,0.086,NaN,NaN
3,dunia.keren3,alhamdulillah ujan.. jadi kereta aman balik en...,0.122,NaN,NaN
4,xfkall,"dapat info malam ini, ada ojol meninggal kelin...",0.199,NaN,NaN
